In [1]:
# https://www.github.com/Nabilphysics
# https://www.nabilbd.com

# !pip install youtube_dl
# !pip install transformers
#!pip install spacy
#!python -m spacy download en_core_web_lg

import youtube_dl
import os

import whisper

In [13]:


def download_ytvid_as_mp3():
    video_url = input("enter url of youtube video:")
    #https://www.youtube.com/watch?v=6KUn7AKmC0Y
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    # Your file location for Ubuntu
    myfile = "youtubeVideo.mp3"
   # If file exists, delete it.
    if os.path.isfile(myfile):
     os.remove(myfile)
    else:
     # If it fails, inform the user.
     print("Error: %s file not found" % myfile)
    
    #filename = f"{video_info['title']}.mp3"
    filename = f"{'youtubeVideo'}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    print("Download complete... {}".format(filename))
download_ytvid_as_mp3()

enter url of youtube video:https://www.youtube.com/watch?v=6KUn7AKmC0Y
[youtube] 6KUn7AKmC0Y: Downloading webpage
[youtube] 6KUn7AKmC0Y: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download] 100% of 7.61MiB in 02:5956KiB/s ETA 00:00
[ffmpeg] Correcting container in "youtubeVideo.mp3"
Download complete... youtubeVideo.mp3


In [4]:
model = whisper.load_model("medium")
#result = model.transcribe('/home/bioforge/Downloads/Spiritual_Authority_1.mp3')
result = model.transcribe('youtubeVideo.mp3')
text = result["text"]
print(text)

 There's a book that sits on my bookshelf that is unlike any other. It invites people to go out and learn about the world in order to better make sense of what was written in it, and in order to prove its own authenticity beyond a shadow of a doubt. In fact, the book encourages its readers to read, learn, ponder, and question things, so much so that the first world revealed from the book was read. But it doesn't stop there. The book goes on to challenge its readers, especially the scientists and scholars among them, with statements such as, we will show them our signs in the universe and in their own selves, until it becomes manifest to them that this is the truth. And these similes we put forward for mankind, but none will understand them except those who have knowledge. The author of this book gives us the total freedom to either believe or disbelieve in what he wrote, but at the same time the author tells us that he also has the freedom to determine our fate after death, with either

In [5]:
#text = result["text"]
#res = len(text.split())
# total no of words
#print ("The number of words in string are : " + str(res))


In [6]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

#punctuation = punctuation + '\n'
#punctuation

def summarize(text, per):
    nlp = spacy.load('en_core_web_lg')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary 

In [7]:
# extractive summary
extractive_summary = summarize(text, 0.15) # 1.0 = 100% so 0.15 = 15%
print(extractive_summary)

The author swears that indeed what is written inside this book is very serious and is not a joke, but the amazing part is that the author swears by two phenomena that at the time the book was revealed, 1400 years ago, were completely unknown to man in any way.In fact, the book encourages its readers to read, learn, ponder, and question things, so much so that the first world revealed from the book was read.The author of this book gives us the total freedom to either believe or disbelieve in what he wrote, but at the same time the author tells us that he also has the freedom to determine our fate after death, with either eternal happiness in paradise or an eternal painful punishment in hell.I think one of the beauties of the book Quran, what makes it so attractive, is that it actually challenges people on a regular throughout the book.Yet not unsurprisingly, the ancient book that I speak of contains numerous proofs in the form of scientific facts and natural wonders that were only disco

In [8]:
# Abstractive Summary Generation

In [9]:
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline



In [10]:
# Pick model
model_name = "google/pegasus-xsum"

# Load pretrained tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)

# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Create tokens
tokens = pegasus_tokenizer(extractive_summary, truncation=True, padding="longest", return_tensors="pt")

# Define summarization pipeline 
summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)

In [11]:
# Create summary 
summary = summarizer(extractive_summary, min_length=20, max_length=150)
summary[0]["summary_text"]

'One of the most famous books that I have ever read is the Quran, the holy book of Islam.'